# Model Name TBD
Combines the time-distributed feature extraction of Convolutional LSTMs with the upsampling and skip connections of a U-Net to convert video-like input features and time-distributed vector metadata into a next frame semantic segmentation map.

### Import Block

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from sklearn.utils import class_weight

#from tensorflow.keras.models import Sequential
#from tensorflow.keras.initializers import Constant
#from skimage.metrics import structural_similarity as ssim
#from skimage.metrics import mean_squared_error
#from math import log10, sqrt

#%matplotlib inline

### Function Definitions

In [2]:
# Loader functions
# Inspiration: https://towardsdatascience.com/writing-custom-keras-generators-fe815d992c5a

def get_2d_input(path):
    # Load array.
    t_2d_input = np.load(path)
    
    return t_2d_input

def get_1d_input(path):
    # Load array.
    t_1d_input = np.load(path)
    
    # Expand dimensions to match model input.
    t_1d_input = tf.expand_dims(tf.expand_dims(t_1d_input, 2), 2)
    
    # Put channel dim at the end.
    t_1d_input = np.moveaxis(t_1d_input, 1, -1)
    
    return t_1d_input

def get_output(path):
    # Load array.
    t_output = np.load(path)
    
    # Put channel dim at the end.
    t_output = np.moveaxis(t_output, 0, -1)
    return t_output

def data_generator(samples, num_samples, batch_size = 64, calculated_sample_weights = None):
    
    while True:
        # Suffle data at the start of each epoch.
        sample_indicies = np.arange(num_samples)
        np.random.shuffle(sample_indicies)
        n = 0
        
        while n + batch_size < num_samples:
            # Get indicies for the batch
            batch_samples  = sample_indicies[n:n + batch_size]
            n += batch_size

            batch_input_2d  = []
            batch_input_1d  = []
            batch_output = [] 
            batch_sample_weights = []

            # Read in each input, perform preprocessing and get labels
            for sample in batch_samples:
                input_2d = get_2d_input(samples.iloc[sample].features_2d)
                input_1d = get_1d_input(samples.iloc[sample].features_1d)
                output = get_output(samples.iloc[sample].labels)
                
                batch_input_2d += [input_2d]
                batch_input_1d += [input_1d]
                batch_output += [output]

                if type(calculated_sample_weights) != type(None):
                    sample_weights = calculated_sample_weights[sample]
                    batch_sample_weights += [sample_weights]
                
            # Return a tuple to feed the network
            batch_x = np.array(batch_input_2d)
            batch_v = np.array(batch_input_1d)
            batch_y = np.array(batch_output)
            
            if type(calculated_sample_weights) == type(None):
                yield([batch_x, batch_v], batch_y)
            else:
                batch_sample_weights = np.array(batch_sample_weights)
                yield([batch_x, batch_v], batch_y, batch_sample_weights)

In [3]:
# Solution for problem with class_weights not working with 3D outputs in tensorflow.
# From: https://github.com/keras-team/keras/issues/3653
def generate_sample_weights(training_data, class_weights): 
    #replaces values for up to 3 classes with the values from class_weights#
    sample_weights = [np.where(y==0,class_weights[0],
                        np.where(y==1,class_weights[1],
                        y)) for y in training_data]
    return np.asarray(sample_weights)

In [4]:
# SSIM/PSNR loss functions.
# Inspiration: https://stackoverflow.com/questions/57357146/use-ssim-loss-function-with-keras
def ssim_loss(y_true, y_pred):
    return 1 - tf.reduce_mean(tf.image.ssim(y_true, y_pred, 1.0))

def psnr_loss(y_true, y_pred):
    return 100 - tf.reduce_mean(tf.image.psnr(y_true, y_pred, 1.0))

### Model Assembly

In [5]:
# Hyperparameters
top_features = 128
condensed_features = 32
upsample_filters = 256
fc_filters = 256

In [6]:
# Inputs broken out by array and vector features.
inputs_2d = layers.Input(shape=((10,32,32,2)))
inputs_1d = layers.Input(shape=((10,1,1,192)))

In [7]:
# Condense 10-day represenations to B * 2 * 2 * C shape.
conv_a = layers.ConvLSTM2D(
    filters=top_features,
    kernel_size=(3, 3),
    padding='same',
    return_sequences=True,
    activation='relu',
    recurrent_dropout = 0,
    name = 'conv_lstm_top'
)(inputs_2d)
conv_b = layers.ConvLSTM2D(
    filters=top_features,
    kernel_size=(2, 2),
    strides = 2,
    return_sequences=True,
    activation='relu',
    recurrent_dropout = 0,
    name = 'conv_lstm_to16'
)(conv_a)
conv_c = layers.ConvLSTM2D(
    filters=top_features,
    kernel_size=(2, 2),
    strides = 2,
    return_sequences=True,
    activation='relu',
    recurrent_dropout = 0,
    name = 'conv_lstm_to8'
)(conv_b)
conv_d = layers.ConvLSTM2D(
    filters=top_features,
    kernel_size=(2, 2),
    strides = 2,
    return_sequences=True,
    activation='relu',
    recurrent_dropout = 0,
    name = 'conv_lstm_to4'
)(conv_c)
conv_e = layers.ConvLSTM2D(
    filters=top_features,
    kernel_size=(2, 2),
    strides = 2,
    return_sequences=True,
    activation='relu',
    recurrent_dropout = 0,
    name = 'conv_lstm_to2'
)(conv_d)
conv_f = layers.ConvLSTM2D(
    filters=64,
    kernel_size=(2, 2),
    strides = 2,
    return_sequences=True,
    activation='relu',
    recurrent_dropout = 0,
    name = 'conv_lstm_to1'
)(conv_e)
conv_a, conv_b, conv_c, conv_d, conv_e, conv_f

(<KerasTensor: shape=(None, 10, 32, 32, 128) dtype=float32 (created by layer 'conv_lstm_top')>,
 <KerasTensor: shape=(None, 10, 16, 16, 128) dtype=float32 (created by layer 'conv_lstm_to16')>,
 <KerasTensor: shape=(None, 10, 8, 8, 128) dtype=float32 (created by layer 'conv_lstm_to8')>,
 <KerasTensor: shape=(None, 10, 4, 4, 128) dtype=float32 (created by layer 'conv_lstm_to4')>,
 <KerasTensor: shape=(None, 10, 2, 2, 128) dtype=float32 (created by layer 'conv_lstm_to2')>,
 <KerasTensor: shape=(None, 10, 1, 1, 64) dtype=float32 (created by layer 'conv_lstm_to1')>)

In [8]:
# Concatenate vectorized features with 10-day fully-connected layer.
vect_cat = tf.keras.layers.Concatenate()([conv_f, inputs_1d])
vect_cat

<KerasTensor: shape=(None, 10, 1, 1, 256) dtype=float32 (created by layer 'concatenate')>

In [9]:
# Condense 10-day representations down to single day.
conv_a_daily = layers.ConvLSTM2D(
    filters=condensed_features,
    kernel_size=(3, 3),
    padding='same',
    return_sequences=False,
    activation='relu',
    recurrent_dropout = 0,
    name = 'conv_lstm_top_daily'
)(conv_a)
conv_b_daily = layers.ConvLSTM2D(
    filters=condensed_features * 2,
    kernel_size=(3, 3),
    padding='same',
    return_sequences=False,
    activation='relu',
    recurrent_dropout = 0,
    name = 'conv_lstm_to16_daily'
)(conv_b)
conv_c_daily = layers.ConvLSTM2D(
    filters=condensed_features * 4,
    kernel_size=(3, 3),
    padding='same',
    return_sequences=False,
    activation='relu',
    recurrent_dropout = 0,
    name = 'conv_lstm_to8_daily'
)(conv_c)
conv_d_daily = layers.ConvLSTM2D(
    filters=condensed_features * 8,
    kernel_size=(3, 3),
    padding='same',
    return_sequences=False,
    activation='relu',
    recurrent_dropout = 0,
    name = 'conv_lstm_to4_daily'
)(conv_d)
conv_e_daily = layers.ConvLSTM2D(
    filters=condensed_features * 16,
    kernel_size=(3, 3),
    padding='same',
    return_sequences=False,
    activation='relu',
    recurrent_dropout = 0,
    name = 'conv_lstm_to2_daily'
)(conv_e)
conv_f_daily = layers.ConvLSTM2D(
    filters=fc_filters,
    kernel_size=(3, 3),
    padding='same',
    return_sequences=False,
    activation='relu',
    recurrent_dropout = 0,
    name = 'conv_lstm_to1_daily'
)(vect_cat)
conv_a_daily, conv_b_daily, conv_c_daily, conv_d_daily, conv_e_daily, conv_f_daily

(<KerasTensor: shape=(None, 32, 32, 32) dtype=float32 (created by layer 'conv_lstm_top_daily')>,
 <KerasTensor: shape=(None, 16, 16, 64) dtype=float32 (created by layer 'conv_lstm_to16_daily')>,
 <KerasTensor: shape=(None, 8, 8, 128) dtype=float32 (created by layer 'conv_lstm_to8_daily')>,
 <KerasTensor: shape=(None, 4, 4, 256) dtype=float32 (created by layer 'conv_lstm_to4_daily')>,
 <KerasTensor: shape=(None, 2, 2, 512) dtype=float32 (created by layer 'conv_lstm_to2_daily')>,
 <KerasTensor: shape=(None, 1, 1, 256) dtype=float32 (created by layer 'conv_lstm_to1_daily')>)

In [10]:
# Upsample layers back to final shape using deconvolution.
x = layers.Conv2D(upsample_filters, (3,3), padding="same", activation='relu', name='UpConv_1_A')(conv_f_daily)
x = layers.Conv2D(upsample_filters, (3,3), padding="same", activation='relu', name='UpConv_1_B')(x)
x = layers.Conv2DTranspose(upsample_filters, (3,3), strides=(2,2), activation='relu', padding='same', name = 'UpConv_to2')(x)
x = layers.BatchNormalization()(x)
x = layers.concatenate([x, conv_e_daily])
x = layers.Conv2D(upsample_filters / 2, (3,3), padding="same", activation='relu', name='UpConv_2_A')(x)
x = layers.Conv2D(upsample_filters / 2, (3,3), padding="same", activation='relu', name='UpConv_2_B')(x)
x = layers.Conv2DTranspose(upsample_filters / 2, (3,3), strides=(2,2), activation='relu', padding='same', name = 'UpConv_to4')(x)
x = layers.BatchNormalization()(x)
x = layers.concatenate([x, conv_d_daily])
x = layers.Conv2D(upsample_filters / 4, (3,3), padding="same", activation='relu', name='UpConv_3_A')(x)
x = layers.Conv2D(upsample_filters / 4, (3,3), padding="same", activation='relu', name='UpConv_3_B')(x)
x = layers.Conv2DTranspose(upsample_filters / 4, (3,3), strides=(2,2), activation='relu', padding='same', name = 'UpConv_to8')(x)
x = layers.BatchNormalization()(x)
x = layers.concatenate([x, conv_c_daily])
x = layers.Conv2D(upsample_filters / 8, (3,3), padding="same", activation='relu', name='UpConv_4_A')(x)
x = layers.Conv2D(upsample_filters / 8, (3,3), padding="same", activation='relu', name='UpConv_4_B')(x)
x = layers.Conv2DTranspose(upsample_filters / 8, (3,3), strides=(2,2), activation='relu', padding='same', name = 'UpConv_to16')(x)
x = layers.BatchNormalization()(x)
x = layers.concatenate([x, conv_b_daily])
x = layers.Conv2D(upsample_filters / 16, (3,3), padding="same", activation='relu', name='UpConv_5_A')(x)
x = layers.Conv2D(upsample_filters / 16, (3,3), padding="same", activation='relu', name='UpConv_5_B')(x)
x = layers.Conv2DTranspose(upsample_filters / 16, (3,3), strides=(2,2), activation='relu', padding='same', name = 'UpConv_to32')(x)
x = layers.BatchNormalization()(x)
x = layers.concatenate([x, conv_a_daily])
x = layers.Conv2D(upsample_filters / 16, (3,3), padding="same", activation='relu')(x)
x = layers.Conv2D(upsample_filters / 16, (3,3), padding="same", activation='relu')(x)
outputs = layers.Conv2D(1, 1, padding='same', activation = 'sigmoid', name = 'outputs')(x)
#outputs = tf.squeeze(x, axis = -1, name = 'squeezed_outputs')

In [11]:
combo_model = tf.keras.Model(inputs = [inputs_2d, inputs_1d], outputs = outputs, name = 'lstm_u_net')

In [12]:
combo_model.summary()

Model: "lstm_u_net"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 10, 32, 32,  0           []                               
                                 2)]                                                              
                                                                                                  
 conv_lstm_top (ConvLSTM2D)     (None, 10, 32, 32,   599552      ['input_1[0][0]']                
                                128)                                                              
                                                                                                  
 conv_lstm_to16 (ConvLSTM2D)    (None, 10, 16, 16,   524800      ['conv_lstm_top[0][0]']          
                                128)                                                     

                                                                                                  
 batch_normalization_3 (BatchNo  (None, 16, 16, 32)  128         ['UpConv_to16[0][0]']            
 rmalization)                                                                                     
                                                                                                  
 conv_lstm_to16_daily (ConvLSTM  (None, 16, 16, 64)  442624      ['conv_lstm_to16[0][0]']         
 2D)                                                                                              
                                                                                                  
 concatenate_4 (Concatenate)    (None, 16, 16, 96)   0           ['batch_normalization_3[0][0]',  
                                                                  'conv_lstm_to16_daily[0][0]']   
                                                                                                  
 UpConv_5_

In [13]:
# Compile model.
opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
#loss_fn = tf.keras.losses.BinaryCrossentropy()
loss_fn = tf.keras.losses.MeanSquaredError()
combo_model.compile(loss=loss_fn, 
                    optimizer=opt, 
                    metrics=[tf.keras.metrics.MeanSquaredError(name='MSE'),
                             tf.keras.metrics.AUC(name='AUC'),
                             ssim_loss,
                             psnr_loss
                            ])

In [14]:
# Test on dummy data to see that shapes look right.
img_batch = tf.zeros([4,10,32,32,2], dtype = 'float32')
vector_batch = tf.zeros([4,10,1,1,192], dtype = 'float32')
combo_model.predict([img_batch, vector_batch]).shape

1/1 [==============================] - 4s 4s/step


(4, 32, 32, 1)

### Data Preparation

In [15]:
# Load metadata on yearly datasets.
df_2017 = pd.read_csv('4fold_super/2017/meta.csv')
df_2018 = pd.read_csv('4fold_super/2018/meta.csv')
df_2019 = pd.read_csv('4fold_super/2019/meta.csv')
df_2020 = pd.read_csv('4fold_super/2020/meta.csv')

# Combine into desired train/val split.
meta_t = pd.concat([df_2017,df_2018,df_2019]).reset_index()
meta_v = df_2020

In [16]:
# Load all labels from the training set into memory to get weights
y_train = []

# Iterate over dataset.
for x in range(0,len(meta_t)):
    y_train.append(np.load(meta_t.iloc[x].labels))

y_train = np.stack(y_train)
y_train = np.minimum(y_train,1)
y_train = tf.expand_dims(y_train, axis = -1).numpy()

# Get class weights for WBCE/MSE.
weights = class_weight.compute_class_weight('balanced',
                                            classes = [0,1],
                                            y = y_train.flatten())
# Examine weights.
weights_dict = {0:weights[0], 1:weights[1]}
weights_dict

{0: 0.5049593185573996, 1: 50.910151537247934}

In [17]:
# Get weights.
calculated_sample_weights = generate_sample_weights(y_train, weights)

# Drop y_train to save memory.
y_train = None

In [18]:
# Data loaders.
batch_size = 32
t_gen = data_generator(meta_t, len(meta_t), batch_size = batch_size, calculated_sample_weights = calculated_sample_weights[:,0])
v_gen = data_generator(meta_v, len(meta_v), batch_size = batch_size)

### Model Training

In [19]:
# Adding callbacks.
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, verbose=1, restore_best_weights=True)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', patience=1, verbose=1)

# Train model.
combo_model.fit(t_gen, 
                   epochs = 100, 
                   verbose = 1, 
                   batch_size = batch_size,
                   validation_data = v_gen,
                   callbacks = [early_stopping, reduce_lr],
                   steps_per_epoch = len(meta_t) // batch_size,
                   validation_steps = len(meta_v) // batch_size
                  )

Epoch 1/400
468/468 [==============================] - 436s 892ms/step - loss: 0.3310 - MSE: 0.2471 - AUC: 0.7669 - ssim_loss: 0.9969 - psnr_loss: 93.4455 - val_loss: 0.3207 - val_MSE: 0.3207 - val_AUC: 0.8624 - val_ssim_loss: 0.9939 - val_psnr_loss: 94.9199 - lr: 1.0000e-04
Epoch 2/400
468/468 [==============================] - 428s 916ms/step - loss: 0.2362 - MSE: 0.1490 - AUC: 0.8970 - ssim_loss: 0.9894 - psnr_loss: 89.6906 - val_loss: 0.3255 - val_MSE: 0.3255 - val_AUC: 0.9090 - val_ssim_loss: 0.9957 - val_psnr_loss: 94.4384 - lr: 1.0000e-04
Epoch 3/400
468/468 [==============================] - 433s 925ms/step - loss: 0.2222 - MSE: 0.1392 - AUC: 0.9116 - ssim_loss: 0.9843 - psnr_loss: 88.8818 - val_loss: 0.1759 - val_MSE: 0.1759 - val_AUC: 0.9152 - val_ssim_loss: 0.9887 - val_psnr_loss: 90.6824 - lr: 1.0000e-04
Epoch 4/400
468/468 [==============================] - 433s 924ms/step - loss: 0.2181 - MSE: 0.1373 - AUC: 0.9162 - ssim_loss: 0.9820 - psnr_loss: 88.7641 - val_loss: 0.078

In [20]:
combo_model.save('Models/CM4_2020')

INFO:tensorflow:Assets written to: Models/CM4_2020\assets


INFO:tensorflow:Assets written to: Models/CM4_2020\assets


In [21]:
#combo_model = tf.keras.models.load_model('Models/CM1_2020', custom_objects = {'ssim_loss': ssim_loss, 'psnr_loss': psnr_loss})

### Model Evaluation

In [22]:
# Get entire validation set.
x_val, v_val, y_val = [],[],[]
for x in range(0,len(meta_v)):
    x_val.append(np.load(meta_v.iloc[x].features_2d))
    v_val.append(np.load(meta_v.iloc[x].features_1d))
    y_val.append(np.load(meta_v.iloc[x].labels))
    
x_val = np.stack(x_val)
v_val = np.stack(v_val)
y_val = np.stack(y_val)

# Dimension wrangling.
v_val = tf.expand_dims(tf.expand_dims(v_val, 2), 2)
y_val = np.moveaxis(y_val, 1, -1)

In [23]:
# Predict on all samples.
all_preds = combo_model.predict([x_val, v_val])

157/157 [==============================] - 33s 197ms/step


In [24]:
# Compute and show set scores.
set_ssim = tf.image.ssim(tf.cast(y_val, dtype='float32'), all_preds, 1.0)
set_psnr = tf.image.psnr(tf.cast(y_val, dtype='float32'), all_preds, 1.0)
set_mse = tf.keras.metrics.mean_squared_error(y_val, all_preds)
print('Model Prediction Report')
print('SSIM:', np.mean(set_ssim))
print('PSNR:', np.mean(set_psnr/100))
print('MSE:', np.mean(set_mse))

Model Prediction Report
SSIM: 0.19108872
PSNR: 0.15968837
MSE: 0.0781578
